<a href="https://colab.research.google.com/github/Usman5-07/DS-Salary/blob/main/Data_Science_Salary_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.1 MB/s eta 0:00:00


In [38]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [13]:
path = '/content/drive/MyDrive/Colab Notebooks/AI, ML, Data Science Salary (2020- 2025).csv'
df = pd.read_csv(path)
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,MI,FT,Customer Success Manager,57000,EUR,60000,NL,50,NL,L
1,2025,SE,FT,Engineer,165000,USD,165000,US,0,US,M
2,2025,SE,FT,Engineer,109000,USD,109000,US,0,US,M
3,2025,SE,FT,Applied Scientist,294000,USD,294000,US,0,US,M
4,2025,SE,FT,Applied Scientist,137600,USD,137600,US,0,US,M


In [19]:
df[df['work_year']==2024]['experience_level'].value_counts()

,count
experience_level,
4,35329
3,19531
1,6183
2,1191


In [17]:
df['experience_level'] = df['experience_level'].replace({'SE':4, 'MI':3, 'EN':1, 'EX':2})

<ipython-input-17-0aa962e366e0>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['experience_level'] = df['experience_level'].replace({'SE':4, 'MI':3, 'EN':1, 'EX':2})


In [24]:
df.head(2)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,3,4,Customer Success Manager,57000,EUR,60000,NL,50,NL,L
1,2025,4,4,Engineer,165000,USD,165000,US,0,US,M


In [20]:
df['employment_type'] = df['employment_type'].replace({'FT':4, 'PT':3, 'FL':2, 'CT':1})

<ipython-input-20-cbfc757cde2e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['employment_type'] = df['employment_type'].replace({'FT':4, 'PT':3, 'FL':2, 'CT':1})


Reason for dropping both these columns is that we have salary in usd and most of the data points in salary_currency are anyway in usd. Also it would make the prediction redundent i.e model not learning any patterns from the data, i.e correlation will be almost 1 if not EQUAL to 1.

In [25]:
df.drop(columns=['salary_currency', 'salary'], inplace=True)

In [26]:
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,3,4,Customer Success Manager,60000,NL,50,NL,L
1,2025,4,4,Engineer,165000,US,0,US,M
2,2025,4,4,Engineer,109000,US,0,US,M
3,2025,4,4,Applied Scientist,294000,US,0,US,M
4,2025,4,4,Applied Scientist,137600,US,0,US,M


In [56]:
df['employee_residence'].nunique()

96

In [37]:
df[df['employee_residence'] == 'LV']['salary_in_usd'].mean()

np.float64(49761.12962962963)

In [35]:
df[df['employee_residence'] == 'US']['salary_in_usd'].mean()

np.float64(163597.51285364782)

In [42]:
import pycountry

def convert_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None  # or 'UNK' if you want to track unknowns

df['employee_residence_alp3'] = df['employee_residence'].apply(convert_to_alpha3)

In [49]:
df[df['employee_residence_alp3']=="QAT"]

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,employee_residence_alp3
82192,2023,3,4,AI Engineer,300000,QA,100,QA,M,QAT


In [44]:
df['employee_residence_alp3'].value_counts().head(10)

,count
employee_residence_alp3,
USA,79705
CAN,3203
GBR,2576
AUS,298
DEU,263
FRA,226
ESP,203
LTU,202
NLD,187


In [54]:
# Choropleth map to check the mean of salary based on employee residence
import plotly.express as px

# Grouping by employee residence and calculating mean salary
salary_by_country = df.groupby('employee_residence_alp3')['salary_in_usd'].mean().reset_index()

# Creating a choropleth map
fig = px.choropleth(
    salary_by_country,
    locations='employee_residence_alp3',
    locationmode='ISO-3',  # Must be ISO Alpha-3 country codes
    color='salary_in_usd',
    color_continuous_scale='Viridis',
    title='Average Salary by Employee Residence',
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
)
fig.show()
fig.write_html("mean_salary_by_residence.html")



AttributeError: 'Figure' object has no attribute 'close'